"""classical_models.ipynb
by: Archie Gertsman (arkadiy2@illinois.edu)
Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license
"""

In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:

import pandas as pd
import numpy as np
from feature_eng import split_trajectories
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_pickle('block4_concat_lane.pkl')
#df = pd.read_pickle('block4_edge_filter.pkl')
df.head()

lat        lon    speed  lon_acc  lat_acc  type  \
file_name id time                                                           
4_1       1  42.00  37.982746  23.732961  11.9046  -0.1145   0.0138  Taxi   
             42.04  37.982746  23.732963  11.8975  -0.1007   0.0147  Taxi   
             42.08  37.982747  23.732964  11.8919  -0.0918   0.0157  Taxi   
             42.12  37.982748  23.732965  11.8871  -0.0869   0.0167  Taxi   
             42.16  37.982748  23.732966  11.8831  -0.0784   0.0176  Taxi   

                    traveled_d  avg_speed   bearing  nearest_edge_start_node  \
file_name id time                                                              
4_1       1  42.00      182.37   9.740748  1.570795                250699362   
             42.04      182.37   9.740748  0.168572                250699362   
             42.08      182.37   9.740748  0.168573                250699362   
             42.12      182.37   9.740748  1.570796                250699362   
             42.16      182.37   9.740748  0.328080                250699362   

                    ...  xtrack_dist  time_stamp  edge_progress_intervals  \
file_name id time   ...                                                     
4_1       1  42.00  ...    -1.883401       42.00                      0.3   
             42.04  ...    -1.980795       42.04                      0.3   
             42.08  ...    -1.937041       42.08                      0.3   
             42.12  ...    -1.893287       42.12                      0.3   
             42.16  ...    -1.941984       42.16                      0.3   

                                edge_id     len  lanes node_veh_dist  \
file_name id time                                                      
4_1       1  42.00  250699362_250699984  97.581    5.4     29.814330   
             42.04  250699362_250699984  97.581    5.4     29.674830   
             42.08  250699362_250699984  97.581    5.4     29.537753   
             42.12  250699362_250699984  97.581    5.4     29.400718   
             42.16  250699362_250699984  97.581    5.4     29.330986   

                    edge_seg  vehicle_density  avg_surr_speed  
file_name id time                                              
4_1       1  42.00       1.0                7       10.464171  
             42.04       1.0                7       10.457843  
             42.08       1.0                7       10.452857  
             42.12       1.0                7       10.448586  
             42.16       1.0                7       10.444986  

[5 rows x 23 columns]

In [4]:
#adding edge_id to the index
df.index = df.reset_index().set_index(['file_name','id','edge_id','time']).index
df.drop(['edge_id'],inplace = True,axis = 1)
df.head()

lat        lon    speed  \
file_name id edge_id             time                                   
4_1       1  250699362_250699984 42.00  37.982746  23.732961  11.9046   
                                 42.04  37.982746  23.732963  11.8975   
                                 42.08  37.982747  23.732964  11.8919   
                                 42.12  37.982748  23.732965  11.8871   
                                 42.16  37.982748  23.732966  11.8831   

                                        lon_acc  lat_acc  type  traveled_d  \
file_name id edge_id             time                                        
4_1       1  250699362_250699984 42.00  -0.1145   0.0138  Taxi      182.37   
                                 42.04  -0.1007   0.0147  Taxi      182.37   
                                 42.08  -0.0918   0.0157  Taxi      182.37   
                                 42.12  -0.0869   0.0167  Taxi      182.37   
                                 42.16  -0.0784   0.0176  Taxi      182.37   

                                        avg_speed   bearing  \
file_name id edge_id             time                         
4_1       1  250699362_250699984 42.00   9.740748  1.570795   
                                 42.04   9.740748  0.168572   
                                 42.08   9.740748  0.168573   
                                 42.12   9.740748  1.570796   
                                 42.16   9.740748  0.328080   

                                        nearest_edge_start_node  ...  dir  \
file_name id edge_id             time                            ...        
4_1       1  250699362_250699984 42.00                250699362  ...    0   
                                 42.04                250699362  ...    0   
                                 42.08                250699362  ...    0   
                                 42.12                250699362  ...    0   
                                 42.16                250699362  ...    0   

                                        xtrack_dist  time_stamp  \
file_name id edge_id             time                             
4_1       1  250699362_250699984 42.00    -1.883401       42.00   
                                 42.04    -1.980795       42.04   
                                 42.08    -1.937041       42.08   
                                 42.12    -1.893287       42.12   
                                 42.16    -1.941984       42.16   

                                        edge_progress_intervals     len  \
file_name id edge_id             time                                     
4_1       1  250699362_250699984 42.00                      0.3  97.581   
                                 42.04                      0.3  97.581   
                                 42.08                      0.3  97.581   
                                 42.12                      0.3  97.581   
                                 42.16                      0.3  97.581   

                                        lanes  node_veh_dist  edge_seg  \
file_name id edge_id             time                                    
4_1       1  250699362_250699984 42.00    5.4      29.814330       1.0   
                                 42.04    5.4      29.674830       1.0   
                                 42.08    5.4      29.537753       1.0   
                                 42.12    5.4      29.400718       1.0   
                                 42.16    5.4      29.330986       1.0   

                                        vehicle_density  avg_surr_speed  
file_name id edge_id             time                                    
4_1       1  250699362_250699984 42.00                7       10.464171  
                                 42.04                7       10.457843  
                                 42.08                7       10.452857  
                                 42.12                7       10.448586  
                                 42.16        

In [5]:
#finding difference between cross track distance between adjacent rows
def __xtrack_dist_diff(df):
    """splits a vehicle trajectory into smaller trajectories of fixed size and removes
    the last (len(df) mod size) riws
    """
    
    df["xtrack_diff"] = df.loc[:,['xtrack_dist']]- df.loc[:,['xtrack_dist']].shift(-1)
    df["xtrack_diff"]=df['xtrack_diff'].fillna(0)
    return df


df = df.groupby(['file_name','id','edge_id'], as_index=False, group_keys=False) \
            .apply(__xtrack_dist_diff)

In [6]:
#Split trajectories with overlap
def split_trajectories_overlap(df, size, overlap):
    """splits each vehicle's trajectory into smaller trajectories of fixed size,
    adding another dimension to the multiindex. Data is truncated to be a multiple
    of `size` in length. 
    Example usage:
        df = csv_to_df('sample.csv')
        df = split_trajectories(df, 3000)
    """
    overlap = 1-overlap
    if overlap == 0:
        return split_trajectories(df,size)
    else:
        df1 = df.groupby(['id','file_name',"edge_id"], as_index=False, group_keys=False) \
                .apply(__split_vehicle, size)

        for i in range(1,int(1/overlap)):
            """ remove x rows from each group and then split the trajectory. 
            eg: say the overlap is 50% with traj_lens = 300, when i = 1, delete first 150 rows and then split trajectory from 
            150-450,450-750 etc
            """
            df2 = df.groupby(['id','file_name',"edge_id"], as_index=False, group_keys=False) \
                .apply(__remove_initial_rows, int(i*overlap*size))
            df2 = df2.groupby(['id','file_name',"edge_id"], as_index=False, group_keys=False) \
                .apply(__split_vehicle, size)
            
            #give different trajectory name for each overlap iteration
            df2.index = df2.index.set_levels(df2.index.levels[3].astype(str) + '_'+str(i), level=3)
            
            #concatenate dataframes one below the other
            df1 = pd.concat([df1,df2],axis = 0)

        return df1

def __remove_initial_rows(df,ele):
    #if ele > number of rows, remedy the error by returning none
    try:
        df1 = df[ele:]
    except TypeError:
        df1 = None
  
    return df1

def __split_vehicle(df, size):
    """splits a vehicle trajectory into smaller trajectories of fixed size and removes
    the last (len(df) mod size) riws
    """
    df2 = df.copy()
    df2['traj'] = None
    df2.loc[::size, 'traj'] = np.arange(len(df2[::size]), dtype=int)
    df2['traj'].ffill(inplace=True)
    df2.set_index('traj', append=True, inplace=True)
    df2 = __truncate_trajectory(df2, size)
    df2 = df2.reorder_levels([0,1,2,4,3])
    return df2


def __truncate_to_multiple(n, m):
    return m * (n // m)

def __truncate_trajectory(traj, size):
    n = len(traj)
    new_len = __truncate_to_multiple(n, size)
    return traj[:new_len]


In [13]:
def agg(df,is_validation_set = False):
    
    df['xtrack_diff_sq'] = df['xtrack_diff']**2
    df_agg = df[np.isin(df['type'], ['Car','Taxi'])] \
        .groupby(['file_name','id', 'edge_id','traj']).agg({
            'xtrack_diff_sq': ['mean','std','skew','max','min',pd.DataFrame.kurt,'sum'],
            'xtrack_diff': ['mean','std','skew',pd.DataFrame.kurt],
            'xtrack_dist': ['mean','std','skew',pd.DataFrame.kurt],
            'avg_surr_speed': ['mean','std','skew',pd.DataFrame.kurt],
            'lanes':['mean'],
            'len':['mean'],
            'speed':['mean','std','skew',pd.DataFrame.kurt,'sum'],
            'vehicle_density': ['mean','std','skew',pd.DataFrame.kurt],
            'lon_acc': ['mean','std','max','min','skew', pd.DataFrame.kurt],
            'lat_acc': ['mean','std','max','min', 'skew', pd.DataFrame.kurt],
            'type': 'first'
        }) \
        
    df_agg.columns = ['_'.join(col) for col in df_agg.columns]
    df_agg.speed_sum = df_agg.speed_sum*0.04
    df_agg.rename(columns={'type_first':'type'}, inplace=True)
    
    if is_validation_set == False:
        g = df_agg.groupby('type')
        df_agg = g.apply(lambda group: group.sample(g.size().min())).reset_index(drop=True)
        X,y = df_agg.drop('type', axis=1), df_agg['type']
        return X,y
    
    else:
        #df_agg = g.apply(lambda group: group.sample(g.size().min()))
        X,y = df_agg.drop('type', axis=1), df_agg['type']
        return X,y


In [38]:
def train_and_accuracy(X_train,y_train,X_test,y_test, model):

    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    a = y_hat==y_test
    
    f = f1_score((y_test == 'Car').astype(int),(y_hat == 'Car').astype(int))
    return len(a[a==True]) / len(y_test),f

def val_voting_accuracy(X_train,y_train,X_val,y_val, model,by_edge = False):

    model.fit(X_train, y_train)
    y_hat = model.predict(X_val)
    if by_edge == False:
        y_hat = pd.DataFrame(index = y_val.index,data = y_hat,columns = ['type'])

        #predicted value for the entire trajectory would be the mode of the predicted labels
        y_hat = y_hat.groupby(['file_name','id']).apply(lambda group: pd.Series.mode(group['type'])[0])
        y_test = y_val.groupby(['file_name','id']).first(['type'])
    else:
        y_hat = pd.DataFrame(index = y_val.index,data = y_hat,columns = ['type'])

        #predicted value for the entire trajectory would be the mode of the predicted labels
        y_hat = y_hat.groupby(['file_name','id','edge_id']).apply(lambda group: pd.Series.mode(group['type'])[0])
        y_test = y_val.groupby(['file_name','id','edge_id']).first(['type'])

    a = y_hat==y_test
   
    f = f1_score((y_test == 'Car').astype(int),(y_hat == 'Car').astype(int))
    return len(a[a==True]) / len(y_test),f


In [47]:

def validation_set(df,test_size):
    """dataframe is split based on their vehicle id's"""
    df1 = df.reset_index()[["file_name",'id','type']].drop_duplicates()
    X = df1[["file_name","id"]]
    y = df1['type']
    X_train,X_test,_,y_test = train_test_split(X, y, test_size=test_size, random_state=4, stratify=y) 
    df_train = df[df.index.droplevel(['time','edge_id']).isin(X_train.set_index(['file_name','id']).index)]
    X_test['type'] = y_test
    g = X_test.groupby('type')
    X_test = g.apply(lambda group: group.sample(g.size().min())).reset_index(drop = True)
    df_test = df[df.index.droplevel(['time','edge_id']).isin(X_test.set_index(['file_name','id']).index)]
    print("No of Cars in validation set %d, No of Taxis in validation set %d" % (sum(X_test['type'] == 'Car'),sum(X_test['type'] == 'Taxi')))
    return df_train,df_test

def process_traj(df2,traj_len,overlap,speed_limit,min_movement_limit,vehicle_density_limit = None):
    """ the trajectories are split into smaller trajectories of fixed size traj_len"""
    df2 = split_trajectories_overlap(df2, traj_len, overlap)
    df3 = df2.reset_index()[["id","file_name","edge_id","traj","speed"]]
    
    #consider only those trajectories which move at a minimum speed of speed_limit for min_movement_limit%
    df3["speed_bool"]= df3["speed"]>speed_limit
    df3 = df3.groupby(["file_name","id","edge_id","traj"]).sum(["speed_bool"])
    df2 = df2[df2.index.droplevel(4).isin(df3[df3.speed_bool >= traj_len*min_movement_limit].index.to_list())]
    
    #filter trajectory by minimum vehicle_density_limit
    if vehicle_density_limit == None:
        return df2
    
    else:
        df3 = df2.reset_index()[["id","file_name","edge_id","traj","vehicle_density"]]
        df3 = df3.groupby(["file_name","id","edge_id","traj"]).mean(["vehicle_density"])
        df2 = df2[df2.index.droplevel(4).isin(df3[df3.vehicle_density >= vehicle_density_limit].index.to_list())]
        
        return df2
    
    

In [48]:

traj_lens = np.arange(300,450, step=50)
models = {
        'Random Forest': Pipeline([('scaler', StandardScaler()), ('rf', RandomForestClassifier())]),
        'AdaBoost':Pipeline([('scaler', StandardScaler()), ('abc', AdaBoostClassifier())]) ,
        'SVM': Pipeline([('scaler', StandardScaler()), ('svc', SVC(max_iter=10000))]) ,
        'Log Regression': Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(max_iter=10000))]) 
    }

df_acc = pd.DataFrame(index=pd.MultiIndex.from_product([models.keys(),['f1_score','accuracy'], ['mean','std']]))
overlap = 0.6
min_movement_limit = 0.75
speed_limit = 0
k = 5
validation_ratio = 0.2
kf = StratifiedKFold(n_splits=k, shuffle=True)
accs = np.zeros(k)
f1 = np.zeros(k)


for traj_len in traj_lens:

    #get length of trajectory in each id
    df_traj_list = df.groupby(['file_name','id','edge_id',]).count()['lat']
    
    #limit trajectory length to greater than traj_len
    df_traj_list = df_traj_list[df_traj_list>= traj_len]
    df2 = df[df.index.droplevel('time').isin(df_traj_list.index.to_list())].copy()
    df_train,df_val = validation_set(df2,validation_ratio)
    
    #process trajectories further
    df_train = process_traj(df_train,traj_len,overlap,speed_limit,min_movement_limit)
    df_val = process_traj(df_val,traj_len,overlap,speed_limit,min_movement_limit)

    #aggregate trajectories
    X,y = agg(df_train)
    #include index for validation dataframes (needed for voting method)
    X_val,y_val = agg(df_val,True)

    #store percent cars and taxis
    print("No of trajectories: ",len(X))
    df_acc.loc[('traj_len','traj_len','total'), (str(traj_len)+'_kfold')] = len(X)
    df_acc.loc[('traj_len','percent','Car'), (str(traj_len)+'_kfold')] = sum(y == 'Car')/len(X)
    df_acc.loc[('traj_len','percent','Taxi'), (str(traj_len)+'_kfold')] = sum(y == 'Taxi')/ len(X)
    
    woedge_count = y_val.reset_index(['traj','edge_id'],drop = True).reset_index().drop_duplicates()
    df_acc.loc[('traj_len','traj_len','total'), (str(traj_len)+'_val_woedge')] = len(woedge_count)
    df_acc.loc[('traj_len','percent','Car'), (str(traj_len)+'_val_woedge')] = sum(woedge_count.type == 'Car')/len(woedge_count)
    df_acc.loc[('traj_len','percent','Taxi'), (str(traj_len)+'_val_woedge')] =sum(woedge_count.type == 'Taxi')/len(woedge_count)
    
    by_edge_count = y_val.reset_index('traj',drop = True).reset_index().drop_duplicates()
    df_acc.loc[('traj_len','traj_len','total'), (str(traj_len)+'_val_by_edge')] = len(by_edge_count)
    df_acc.loc[('traj_len','percent','Car'), (str(traj_len)+'_val_by_edge')] = sum(by_edge_count.type == 'Car')/len(by_edge_count)
    df_acc.loc[('traj_len','percent','Taxi'), (str(traj_len)+'_val_by_edge')] = sum(by_edge_count.type == 'Taxi')/len(by_edge_count)
    
    # fit different models
    for name, model in models.items():
        for i, (train_index, test_index) in enumerate(kf.split(X,y)):
            
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            accs[i],f1[i] = train_and_accuracy(X_train, y_train,X_test,y_test, model)
        
        df_acc.loc[(name, 'accuracy','mean'), (str(traj_len)+'_kfold')] = round(100*accs.mean(), 3)
        df_acc.loc[(name, 'accuracy','std'), (str(traj_len)+'_kfold')] = round(100*accs.std(), 3)
        df_acc.loc[(name, 'f1_score','mean'),(str(traj_len)+'_kfold')] = round(100*f1.mean(), 3)
        df_acc.loc[(name, 'f1_score','std'), (str(traj_len)+'_kfold')] = round(100*f1.std(), 3)

        val_accs,val_f1 = val_voting_accuracy(X, y,X_val,y_val, model)
        df_acc.loc[(name, 'accuracy','mean'), (str(traj_len)+'_val_woedge')] = round(100*val_accs, 3)
        df_acc.loc[(name, 'f1_score','mean'),(str(traj_len)+'_val_woedge')] = round(100*val_f1, 3)
        
        val_accs,val_f1 = val_voting_accuracy(X, y,X_val,y_val, model,by_edge = True)
        df_acc.loc[(name, 'accuracy','mean'), (str(traj_len)+'_val_by_edge')] = round(100*val_accs, 3)
        df_acc.loc[(name, 'f1_score','mean'),(str(traj_len)+'_val_by_edge')] = round(100*val_f1, 3)

        print(name, 'complete.')

    print('trajectory length', traj_len, 'complete.')

df_acc

No of Cars in validation set 105, No of Taxis in validation set 105
No of trajectories:  2834
Random Forest complete.
AdaBoost complete.
SVM complete.
Log Regression complete.
trajectory length 300 complete.
No of Cars in validation set 95, No of Taxis in validation set 95
No of trajectories:  2058
Random Forest complete.
AdaBoost complete.
SVM complete.
Log Regression complete.
trajectory length 350 complete.
No of Cars in validation set 85, No of Taxis in validation set 85
No of trajectories:  1536
Random Forest complete.
AdaBoost complete.
SVM complete.
Log Regression complete.
trajectory length 400 complete.


300_kfold  300_val_woedge  300_val_by_edge  \
Random Forest  f1_score mean      61.249       61.611000        58.824000   
                        std        2.346             NaN              NaN   
               accuracy mean      62.173       60.099000        56.818000   
                        std        1.546             NaN              NaN   
AdaBoost       f1_score mean      58.278       61.165000        60.256000   
                        std        0.946             NaN              NaN   
               accuracy mean      60.339       60.591000        59.740000   
                        std        1.317             NaN              NaN   
SVM            f1_score mean      55.334       54.737000        54.417000   
                        std        2.823             NaN              NaN   
               accuracy mean      61.820       57.635000        58.117000   
                        std        2.095             NaN              NaN   
Log Regression f1_score mean      55.032       49.746000        50.676000   
                        std        1.191             NaN              NaN   
               accuracy mean      57.869       51.232000        52.597000   
                        std        1.304             NaN              NaN   
traj_len       traj_len total   2834.000      203.000000       308.000000   
               percent  Car        0.500        0.487685         0.496753   
                        Taxi       0.500        0.512315         0.503247   

                               350_kfold  350_val_woedge  350_val_by_edge  \
Random Forest  f1_score mean      62.608       57.923000        54.980000   
                        std        2.720             NaN              NaN   
               accuracy mean      64.189       56.497000        54.980000   
                        std        1.723             NaN              NaN   
AdaBoost       f1_score mean      61.518       58.140000        56.432000   
                        std        1.310             NaN              NaN   
               accuracy mean      63.314       59.322000        58.167000   
                        std        1.027             NaN              NaN   
SVM            f1_score mean      57.992       52.500000        47.847000   
                        std        3.813             NaN              NaN   
               accuracy mean      63.267       57.062000        56.574000   
                        std        2.905             NaN              NaN   
Log Regression f1_score mean      53.678       50.314000        49.778000   
                        std        3.262             NaN              NaN   
               accuracy mean      58.065       55.367000        54.980000   
                        std        1.797             NaN              NaN   
traj_len       traj_len total   2058.000      177.000000       251.000000   
               percent  Car        0.500        0.491525         0.478088   
                        Taxi       0.500        0.508475         0.521912   

                               400_kfold  400_val_woedge  400_val_by_edge  
Random Forest  f1_score mean      58.631       56.757000        53.333000  
                        std        4.567             NaN              NaN  
               accuracy mean      60.413       56.463000        54.950000  
                        std        3.886             NaN              NaN  
AdaBoost       f1_score mean      56.080       51.471000        52.747000  
                        std        2.463             NaN              NaN  
               accuracy mean      58.141       55.102000        57.426000  
                        std        3.052             NaN              NaN  
SVM            f1_score mean      52.883       53.226000        51.534000  
                        std        4.682             NaN              NaN  
               accuracy mean      59.830       60.544000        60.891000  
                        std        3.

In [ ]:

traj_lens = np.arange(200,450, step=50)
df_acc = pd.DataFrame(index=pd.MultiIndex.from_product([models.keys(),['f1_score','accuracy'], ['mean','std']]))
overlap = 0.6
min_movement_limit = 0.75
speed_limit = 0
k = 5
validation_ratio = 0.2
kf = StratifiedKFold(n_splits=k, shuffle=True)
accs = np.zeros(k)
f1 = np.zeros(k)

for j in range(0,5):
    
    for traj_len in traj_lens:

        #get length of trajectory in each id
        df_traj_list = df.groupby(['file_name','id','edge_id',]).count()['lat']

        #limit trajectory length to greater than traj_len
        df_traj_list = df_traj_list[df_traj_list>= traj_len]
        df2 = df[df.index.droplevel('time').isin(df_traj_list.index.to_list())].copy()
        df_train,df_val = validation_set(df2,validation_ratio)

        #process trajectories further
        df_train = process_traj(df_train,traj_len,overlap,speed_limit,min_movement_limit,j)
        df_val = process_traj(df_val,traj_len,overlap,speed_limit,min_movement_limit,j)

        #aggregate trajectories
        X,y = agg(df_train)
        #include index for validation dataframes (needed for voting method)
        X_val,y_val = agg(df_val,True)

        #store percent cars and taxis
        print("No of trajectories: ",len(X))
        df_acc.loc[('traj_len','traj_len','total'), (str(traj_len)+'_kfold')] = len(X)
        df_acc.loc[('traj_len','percent','Car'), (str(traj_len)+'_kfold')] = sum(y == 'Car')/len(X)
        df_acc.loc[('traj_len','percent','Taxi'), (str(traj_len)+'_kfold')] = sum(y == 'Taxi')/ len(X)

        woedge_count = y_val.reset_index(['traj','edge_id'],drop = True).reset_index().drop_duplicates()
        df_acc.loc[('traj_len','traj_len','total'), (str(traj_len)+'_val_woedge')] = len(woedge_count)
        df_acc.loc[('traj_len','percent','Car'), (str(traj_len)+'_val_woedge')] = sum(woedge_count.type == 'Car')/len(woedge_count)
        df_acc.loc[('traj_len','percent','Taxi'), (str(traj_len)+'_val_woedge')] =sum(woedge_count.type == 'Taxi')/len(woedge_count)

        by_edge_count = y_val.reset_index('traj',drop = True).reset_index().drop_duplicates()
        df_acc.loc[('traj_len','traj_len','total'), (str(traj_len)+'_val_by_edge')] = len(by_edge_count)
        df_acc.loc[('traj_len','percent','Car'), (str(traj_len)+'_val_by_edge')] = sum(by_edge_count.type == 'Car')/len(by_edge_count)
        df_acc.loc[('traj_len','percent','Taxi'), (str(traj_len)+'_val_by_edge')] = sum(by_edge_count.type == 'Taxi')/len(by_edge_count)

        # fit different models
        for name, model in models.items():
            for i, (train_index, test_index) in enumerate(kf.split(X,y)):

                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                accs[i],f1[i] = train_and_accuracy(X_train, y_train,X_test,y_test, model)

            df_acc.loc[(name, 'accuracy','mean'), (str(traj_len)+'_kfold')] = round(100*accs.mean(), 3)
            df_acc.loc[(name, 'accuracy','std'), (str(traj_len)+'_kfold')] = round(100*accs.std(), 3)
            df_acc.loc[(name, 'f1_score','mean'),(str(traj_len)+'_kfold')] = round(100*f1.mean(), 3)
            df_acc.loc[(name, 'f1_score','std'), (str(traj_len)+'_kfold')] = round(100*f1.std(), 3)

            val_accs,val_f1 = val_voting_accuracy(X, y,X_val,y_val, model)
            df_acc.loc[(name, 'accuracy','mean'), (str(traj_len)+'_val_woedge')] = round(100*val_accs, 3)
            df_acc.loc[(name, 'f1_score','mean'),(str(traj_len)+'_val_woedge')] = round(100*val_f1, 3)

            val_accs,val_f1 = val_voting_accuracy(X, y,X_val,y_val, model,by_edge = True)
            df_acc.loc[(name, 'accuracy','mean'), (str(traj_len)+'_val_by_edge')] = round(100*val_accs, 3)
            df_acc.loc[(name, 'f1_score','mean'),(str(traj_len)+'_val_by_edge')] = round(100*val_f1, 3)

            print(name, 'complete.')
            
        print('trajectory length', traj_len, 'complete.')

    print("vehicle density >= ",j)
    print(df_acc)

No of Cars in validation set 127, No of Taxis in validation set 127
